# Explore here

In [15]:
import kaggle
import zipfile
import os

dataset_identifier = 'iphone-customer-reviews-nlp/data?select=iphone.csv'

# Download the dataset from Kaggle
kaggle.api.dataset_download_files(dataset_identifier, path='datasets/', unzip=True)

print(f'{dataset_identifier} dataset downloaded and unzipped into the datasets folder.')


Dataset URL: https://www.kaggle.com/datasets/iphone-customer-reviews-nlp/data?select=iphone.csv


ApiException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json', 'Date': 'Mon, 23 Sep 2024 04:23:14 GMT', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Origin': '*', 'Set-Cookie': 'ka_sessionid=2e88c6a953e29bb12384e5de3c2410c8; max-age=2626560; path=/, GCLB=CP2f6qaf07rv8wEQAw; path=/; HttpOnly', 'Vary': 'Accept-Encoding', 'X-Kaggle-MillisecondsElapsed': '18', 'X-Kaggle-RequestId': 'cc19a12c52dfab8a6057f01f13900264', 'X-Kaggle-ApiVersion': '1.6.17', 'X-Kaggle-HubVersion': '0.3.0', 'X-Frame-Options': 'SAMEORIGIN', 'Strict-Transport-Security': 'max-age=63072000; includeSubDomains; preload', 'Content-Security-Policy': "object-src 'none'; script-src 'nonce-9iPblosKGAw7rG7Dx87qjQ==' 'report-sample' 'unsafe-inline' 'unsafe-eval' 'strict-dynamic' https: http:; base-uri 'none'; report-uri https://csp.withgoogle.com/csp/kaggle/20201130; frame-src 'self' https://www.kaggleusercontent.com https://www.youtube.com/embed/ https://pol

In [2]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split

# Load your dataset
file_path = '../data/raw/iphone.csv'  # Adjust to your local path
iphone_reviews = pd.read_csv(file_path)

# Clean text (remove special characters, make lowercase)
def clean_text(text):
    text = re.sub(r'[^A-Za-z\s]', '', str(text))  # Remove non-alphabet characters
    return text.lower()

# Apply cleaning to review descriptions
iphone_reviews['reviewDescription'] = iphone_reviews['reviewDescription'].fillna('')
iphone_reviews['cleaned_review'] = iphone_reviews['reviewDescription'].apply(clean_text)

# Create sentiment labels based on ratingScore
def label_sentiment(rating):
    if rating <= 2:
        return 'negative'
    elif rating == 3:
        return 'neutral'
    else:
        return 'positive'

iphone_reviews['sentiment'] = iphone_reviews['ratingScore'].apply(label_sentiment)

# Split the data into training and test sets
X = iphone_reviews['cleaned_review']
y = iphone_reviews['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the training data, and transform the test data
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Train Logistic Regression model
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_tfidf, y_train)

# Make predictions and evaluate
y_pred_logreg = logreg.predict(X_test_tfidf)
print("Logistic Regression Performance:")
print(classification_report(y_test, y_pred_logreg))


Logistic Regression Performance:
              precision    recall  f1-score   support

    negative       0.70      0.62      0.66       130
     neutral       1.00      0.04      0.07        53
    positive       0.84      0.97      0.90       430

    accuracy                           0.81       613
   macro avg       0.85      0.54      0.54       613
weighted avg       0.82      0.81      0.78       613



In [5]:
from sklearn.naive_bayes import MultinomialNB

# Train Naive Bayes model
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

# Make predictions and evaluate
y_pred_nb = nb.predict(X_test_tfidf)
print("Naive Bayes Performance:")
print(classification_report(y_test, y_pred_nb))


Naive Bayes Performance:
              precision    recall  f1-score   support

    negative       0.89      0.25      0.40       130
     neutral       0.00      0.00      0.00        53
    positive       0.74      0.99      0.85       430

    accuracy                           0.75       613
   macro avg       0.54      0.42      0.41       613
weighted avg       0.71      0.75      0.68       613



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [6]:
from sklearn.svm import SVC

# Train SVM model
svm = SVC()
svm.fit(X_train_tfidf, y_train)

# Make predictions and evaluate
y_pred_svm = svm.predict(X_test_tfidf)
print("SVM Performance:")
print(classification_report(y_test, y_pred_svm))


SVM Performance:
              precision    recall  f1-score   support

    negative       0.74      0.62      0.68       130
     neutral       1.00      0.13      0.23        53
    positive       0.84      0.97      0.90       430

    accuracy                           0.82       613
   macro avg       0.86      0.57      0.60       613
weighted avg       0.83      0.82      0.80       613



In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Initialize and train the Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_tfidf, y_train)

# Make predictions and evaluate
y_pred_rf = rf.predict(X_test_tfidf)
print("Random Forest Performance:")
print(classification_report(y_test, y_pred_rf))


Random Forest Performance:
              precision    recall  f1-score   support

    negative       0.74      0.52      0.61       130
     neutral       0.92      0.21      0.34        53
    positive       0.82      0.97      0.89       430

    accuracy                           0.81       613
   macro avg       0.83      0.57      0.61       613
weighted avg       0.81      0.81      0.78       613



In [9]:
# Collect model performance results for comparison
from sklearn.metrics import accuracy_score

models = {
    'Logistic Regression': logreg,
    'Naive Bayes': nb,
    'SVM': svm,
    'Random Forest': rf
}

# Evaluate and print accuracy of each model
for name, model in models.items():
    y_pred = model.predict(X_test_tfidf)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred))


Logistic Regression Accuracy: 0.8140
              precision    recall  f1-score   support

    negative       0.70      0.62      0.66       130
     neutral       1.00      0.04      0.07        53
    positive       0.84      0.97      0.90       430

    accuracy                           0.81       613
   macro avg       0.85      0.54      0.54       613
weighted avg       0.82      0.81      0.78       613

Naive Bayes Accuracy: 0.7504
              precision    recall  f1-score   support

    negative       0.89      0.25      0.40       130
     neutral       0.00      0.00      0.00        53
    positive       0.74      0.99      0.85       430

    accuracy                           0.75       613
   macro avg       0.54      0.42      0.41       613
weighted avg       0.71      0.75      0.68       613

SVM Accuracy: 0.8238
              precision    recall  f1-score   support

    negative       0.74      0.62      0.68       130
     neutral       1.00      0.13      0.2

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [10]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Define a parameter grid to search through
param_grid = {
    'C': [0.1, 1, 10, 100],  # Penalty parameter
    'kernel': ['linear', 'rbf'],  # Linear and Radial Basis Function kernels
    'gamma': ['scale', 'auto']  # Kernel coefficient
}

# Set up the GridSearchCV to find the best combination of hyperparameters
grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', verbose=1)

# Fit the grid search model
grid_search.fit(X_train_tfidf, y_train)

# Print the best hyperparameters found
print("Best Parameters: ", grid_search.best_params_)

# Evaluate the tuned model
y_pred_svm_tuned = grid_search.best_estimator_.predict(X_test_tfidf)
print("Tuned SVM Performance:")
print(classification_report(y_test, y_pred_svm_tuned))


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters:  {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Tuned SVM Performance:
              precision    recall  f1-score   support

    negative       0.74      0.71      0.72       130
     neutral       0.92      0.21      0.34        53
    positive       0.87      0.97      0.92       430

    accuracy                           0.85       613
   macro avg       0.84      0.63      0.66       613
weighted avg       0.85      0.85      0.83       613



In [11]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))  


In [12]:
svm = SVC(class_weight='balanced')


Dataset Characteristics:

The dataset consists of iPhone reviews from Amazon, with three sentiment classes: positive, neutral, and negative.
The reviews are largely skewed towards the positive class (with most ratings being 4 or 5 stars), while neutral and negative reviews are relatively fewer in number.
This class imbalance presents a challenge, especially for predicting neutral sentiment, which the model struggles with.
Model Performance:

After tuning the SVM model using a radial basis function (RBF) kernel, the overall accuracy improved to 85%. This indicates that the model is quite effective at classifying the reviews into the appropriate sentiment categories.
The model performs exceptionally well in predicting positive sentiment, achieving a high F1-score of 0.92 and recall of 0.97.
Predictions for negative sentiment also improved, with a reasonable F1-score of 0.72. However, there’s still room to enhance recall, suggesting that some negative reviews may not be correctly identified.
The neutral sentiment is where the model performs weakest, with an F1-score of 0.34. This is likely due to the class imbalance and the inherent difficulty in distinguishing neutral reviews from either positive or negative ones.
Model Implications:

The model is highly effective for classifying positive reviews, making it suitable for applications that prioritize identifying strong customer satisfaction.
It performs moderately well in detecting negative reviews, which could still be useful for flagging potentially dissatisfied customers. However, improvements in recall could ensure that more negative reviews are caught.
The inability to accurately detect neutral sentiment means that the model may struggle to identify mixed or balanced feedback, where users may express both pros and cons. In real-world applications, this could lead to an overemphasis on extreme sentiments (positive or negative).
Use Case Applications:

Customer Feedback Analysis: The model is well-suited for analyzing customer feedback at scale, particularly for identifying highly satisfied or dissatisfied customers.
Market Insights: Companies can use the model to understand the distribution of sentiment in product reviews, helping them gauge product reception and improve areas of concern based on negative reviews.
Customer Service Prioritization: Businesses could use the model to prioritize handling negative reviews quickly, while understanding that neutral feedback may require further refinement in the model to be detected accurately.